In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install langchain langchain-community langchain-nvidia-ai-endpoints 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2,

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [4]:
from kaggle_secrets import UserSecretsClient
nvidia_api_key = UserSecretsClient().get_secret("NVIDIA_API_KEY")
nvidia_model_name = UserSecretsClient().get_secret("NVIDIA_MODEL_NAME")

In [5]:
generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a twitter techie influencer assistant tasked with weiting excellent twitter posts."
            "Generate the best twitter post possible for the user's request."
            "If the user provides critique , respond with a revised version of your previous attempts.",
        
        ),
        MessagesPlaceholder(variable_name = "messages"),
    ]

)

In [6]:
 reflection_prompt = ChatPromptTemplate.from_messages(
     [
         (
             "system",
             "You are a viral twitter influencer grading a tweet. Generate critique and recomendations for the user's tweet."
             "Always provide detailed recommendations, including requests for length, virality, style, etc.",
         ),
         MessagesPlaceholder(variable_name = "messages"),
     ]
 )

In [7]:
llm = ChatNVIDIA(
    model = nvidia_model_name,
    api_key = nvidia_api_key,
)

In [8]:
generation_chain = generation_prompt | llm

reflection_chain = reflection_prompt | llm

In [9]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66


In [10]:
from typing import List, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph

In [11]:
graph = MessageGraph()

REFLECT = "reflect"
GENERATE = "generate"

/tmp/ipykernel_13/4047370416.py:1: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


In [12]:
def generate_node(state):
    return generation_chain.invoke({
        "messages": state
    }
    
    )

def reflect_node(messages):
    response = reflection_chain.invoke({
        "messages": messages
    }
        
    )
    return [HumanMessage(content = response.content)]

In [13]:
graph.add_node(GENERATE, generate_node)
graph.add_node(REFLECT, reflect_node)

In [14]:
graph.set_entry_point(GENERATE)

In [15]:
def should_continue(state):
    if (len(state) > 4):
        return END
    return REFLECT

In [16]:
graph.add_conditional_edges(GENERATE, should_continue)


In [17]:
graph.add_edge(REFLECT, GENERATE)

In [18]:
app = graph.compile()

In [19]:
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.5 MB/s eta 0:00:00


In [20]:
print(app.get_graph().print_ascii())

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
+----------+   
| generate |   
+----------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   
None


In [21]:
print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__(<p>__start__</p>)
	generate(generate)
	reflect(reflect)
	__end__(<p>__end__</p>)
	__start__ --> generate;
	generate --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [22]:
secret_value_0 = UserSecretsClient().get_secret("LANGSMITH_API_KEY")
secret_value_1 = UserSecretsClient().get_secret("LANGSMITH_ENDPOINT")
secret_value_2 = UserSecretsClient().get_secret("LANGSMITH_PROJECT")
secret_value_3 = UserSecretsClient().get_secret("LANGSMITH_TRACING")
secret_value_4 = UserSecretsClient().get_secret("NVIDIA_API_KEY")
secret_value_5 = UserSecretsClient().get_secret("NVIDIA_MODEL_NAME")

In [23]:
response = app.invoke(HumanMessage(content = "Ai agents taking over content creation"))



In [24]:
print(response)

[HumanMessage(content='Ai agents taking over content creation', additional_kwargs={}, response_metadata={}, id='fe4092cd-8be2-415f-80c6-ff63c73f6ce7'), AIMessage(content='Here\'s a Twitter post for the topic "AI agents taking over content creation":\n\n**Initial Post**\n\n"The AI revolution has arrived! As machines become increasingly capable of creating high-quality content, the lines between human and artificial creativity start to blur. Will AI agents take over content creation? Should we embrace the change or fear the machines? #AIContent #FutureOfWork"\n\nLet me know if you\'d like me to revise or expand on this post!', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Here\'s a Twitter post for the topic "AI agents taking over content creation":\n\n**Initial Post**\n\n"The AI revolution has arrived! As machines become increasingly capable of creating high-quality content, the lines between human and artificial creativity start to blur. Will AI agents take 